In [3]:
from pprint import pprint
import sys
import os, json, re, requests
import ckanapi.errors
from ckanapi.errors import NotFound
import pandas as pd
import random

from pathlib import Path
import json

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
from ckan_migration import Migrate

In [207]:
ORIGINAL_CKAN_URL = 'http://staging.basedosdados.org'

In [210]:
def get_package_show(package):
    
    api_url = ORIGINAL_CKAN_URL + f'/api/3/action/package_show?id={package}'
    return requests.get(api_url, verify=False).json()['result']

def download_packages():

    api_url = ORIGINAL_CKAN_URL + '/api/3/action/package_list'
    packages = requests.get(api_url, verify=False).json()['result']

    for p in packages:
        json.dump(get_package_show(p), open(f'packages/{p}', 'w'))

In [211]:
download_packages()

/opt/homebrew/Caskroom/miniforge/base/envs/sandbox/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'staging.basedosdados.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/sandbox/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'staging.basedosdados.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/sandbox/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'staging.basedosdados.org'. Adding certificate verification is strongly advised. See: https://urllib

In [6]:
def package_update(package_path, validate_only=True):
    
    package = json.load(open(package_path))
    migrated = migrate(package)
    
    if validate(migrated):
        raise Exception(res)
    
    if not validate_only:
        try:
            basedosdados.action.package_update(**migrated)
        except NotFound:
            print(package_path)

In [235]:
validated_list = []

In [317]:
datas_list = ['2008-2019', '1987-88, 1995-96, 2002-03, 2008-09', 'Atual',
             '-', '2019', '1978, 1992-2019'
             ]

In [326]:
def _treat_datas(datas):
    
    final = []
    
    datas = datas.lower()
    
    if datas in ['atual', '-']:
        return None
    else:
        print(datas)
        for d_range in datas.split(','):
            if '-' in d_range:
                rang = [r.strip() for r in d_range.split('-')]
                if len(rang[1]) == 2:
                    rang[1] = rang[0][:2]  + rang[1]
#                 print('d_range', d_range, 
#                       list(range(int(rang[0]), int(rang[1]) + 1)))
                final.extend(
                    list(range(int(rang[0]), int(rang[1]) + 1)))
            else:
#                 print('year', [int(d_range)])
                final.extend([int(d_range)])
                
    return final

In [633]:
validated_list = []

In [ ]:
'world-values-survey'

In [642]:
package = json.loads(open('packages/world-values-survey').read())
a = Migrate(package)
print(a.package_dict, '\n')
a.migrate()
val = a.validate()

{'id': 'ad95e661-7d0e-4bb0-8ce4-c5ff6e8a652e', 'name': 'world-values-survey', 'title': 'World Values Survey (WVS)', 'version': '', 'url': '', 'author': '', 'author_email': '', 'maintainer': '', 'maintainer_email': '', 'notes': 'O World Values Survey (WVS), um projeto de pesquisa global, explora os valores e crenças das pessoas, como elas mudam ao longo do tempo e que impacto social e político elas têm. Desde 1981, uma rede mundial de cientistas sociais realizou pesquisas nacionais representativas como parte do WVS em quase 100 países.\r\n\r\nO WVS mede, monitora e analisa: apoio à democracia, tolerância a estrangeiros e minorias étnicas, apoio à igualdade de gênero, papel da religião e mudança nos níveis de religiosidade, impacto da globalização, atitudes em relação ao meio ambiente, trabalho, família, política, identidade nacional, cultura, diversidade, insegurança e bem-estar subjetivo.', 'license_id': 'cc-by', 'type': 'dataset', 'owner_org': '2cb2eef5-f511-41d9-bcf3-1c84f8d15366', '

In [679]:
for p in Path('packages').glob('*'):
    if p.name not in validated_list:
        package = json.loads(p.open().read())
        a = Migrate(package)
        a.migrate()
        val = a.validate()
        if isinstance(val, list):
            print(val, '\n')
            print(a.package_dict, '\n')
            
            break
        validated_list.append(p.name)

[{'loc': ['resources', 0, 'ExternalLink', 'has_structured_data'], 'msg': "value is not a valid enumeration member; permitted: 'yes', 'no'", 'type': 'type_error.enum', 'ctx': {'enum_values': ['yes', 'no']}}, {'loc': ['resources', 0, 'ExternalLink', 'entity', 0], 'msg': "value is not a valid enumeration member; permitted: 'continent', 'country', 'region', 'state', 'county', 'district', 'municipality', 'city', 'village', 'neighborhood', 'zip_code', 'census_tract'", 'type': 'type_error.enum', 'ctx': {'enum_values': ['continent', 'country', 'region', 'state', 'county', 'district', 'municipality', 'city', 'village', 'neighborhood', 'zip_code', 'census_tract']}}, {'loc': ['resources', 0, 'ExternalLink', 'entity', 0], 'msg': "value is not a valid enumeration member; permitted: 'person', 'household', 'name', 'animal', 'plant'", 'type': 'type_error.enum', 'ctx': {'enum_values': ['person', 'household', 'name', 'animal', 'plant']}}, {'loc': ['resources', 0, 'ExternalLink', 'entity', 0], 'msg': "va

In [672]:
len(validated_list)

186